<a href="https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_multilenguaje_cuaderno_de_entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="pink"> **Cuaderno de entrenamiento de [Piper.](https://github.com/rhasspy/piper)**
## ![Piper logo](https://contribute.rhasspy.org/img/logo.png)

---

- Cuaderno creado por [rmcpantoja](http://github.com/rmcpantoja)
- Colaborador y traductor: [Xx_Nessu_xX](https://fakeyou.com/profile/Xx_Nessu_xX)
- [Cuaderno original inglés.](https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_multilingual_training_notebook.ipynb#scrollTo=dOyx9Y6JYvRF)

---

# Notas:

- <font color="orange">**Las cosas en naranja significa que son importantes.**

# <font color="pink">🔧 ***Primeros pasos.*** 🔧

In [ ]:
#@markdown ## <font color="pink"> **Google Colab Anti-Disconnect.** 🔌
#@markdown ---
#@markdown #### Evita la desconexión automática. Aún así, se desconectará después de <font color="orange">**6 a 12 horas**</font>.

import IPython
js_code = '''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''
display(IPython.display.Javascript(js_code))

In [ ]:
#@markdown ## <font color="pink"> **Comprueba la GPU.** 👁️
#@markdown ---
#@markdown #### Una GPU de mayor capacidad puede aumentar la velocidad de entrenamiento. Por defecto, tendrás una <font color="orange">**Tesla T4**</font>.
!nvidia-smi

In [ ]:
#@markdown # <font color="pink"> **Monta tu Google Drive.** 📂
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@markdown # <font color="pink"> **Instalar software.** 📦

#@markdown ####En esta celda se instalará el sintetizador y sus dependencias necesarias para ejecutar el entrenamiento. (Esto puede llevar un rato.)

#@markdown #### <font color="orange">**¿Quieres usar el parche?**
#@markdown El parche ofrece la posibilidad de exportar archivos de audio a la carpeta de salida y guardar un único modelo durante el entrenamiento.
usepatch = True #@param {type:"boolean"}
#@markdown ---
# clone:
!git clone -q https://github.com/rmcpantoja/piper
%cd /content/piper/src/python
!wget -q "https://raw.githubusercontent.com/coqui-ai/TTS/dev/TTS/bin/resample.py"
!pip install -q -r requirements.txt
!pip install -q torchtext==0.12.0
!pip install -q torchvision==0.12.0
# fixing recent compativility isswes:
!pip install -q torchaudio==0.11.0 torchmetrics==0.11.4
!bash build_monotonic_align.sh
!apt-get install -q espeak-ng
# download patches:
print("Downloading patch...")
!gdown -q "1EWEb7amo1rgFGpBFfRD4BKX3pkjVK1I-" -O "/content/piper/src/python/patch.zip"
!unzip -o -q "patch.zip"
%cd /content

# <font color="pink"> 🤖 ***Entrenamiento.*** 🤖

In [ ]:
#@markdown # <font color="pink"> **1. Extraer dataset.** 📥
#@markdown ####Importante: los audios deben estar en formato <font color="orange">**wav, (16000 o 22050hz, 16-bits, mono), y, por comodidad, numerados. Ejemplo:**

#@markdown * <font color="orange">**1.wav**</font>
#@markdown * <font color="orange">**2.wav**</font>
#@markdown * <font color="orange">**3.wav**</font>
#@markdown * <font color="orange">**.....**</font>

#@markdown ---

%cd /content
!mkdir /content/dataset
%cd /content/dataset
!mkdir /content/dataset/wavs
#@markdown ### Ruta del dataset para descomprimir:
zip_path = "/content/drive/MyDrive/wavs.zip" #@param {type:"string"}
!unzip "{zip_path}" -d /content/dataset/wavs
#@markdown ---

In [ ]:
#@markdown # <font color="pink"> **2. Cargar el archivo de transcripción.** 📝
#@markdown ---
#@markdown ####<font color="orange">**Importante: la transcripción significa escribir lo que dice el personaje en cada uno de los audios, y debe tener la siguiente estructura:**

#@markdown ##### <font color="orange">Para un conjunto de datos de un solo hablante:
#@markdown * wavs/1.wav|Esto dice el personaje en el audio 1.
#@markdown * wavs/2.wav|Este, el texto que dice el personaje en el audio 2.
#@markdown * ...

#@markdown ##### <font color="orange">Para un conjunto de datos de varios hablantes:

#@markdown * wavs/speaker1audio1.wav|speaker1|Esto es lo que dice el primer hablante.
#@markdown * wavs/speaker1audio2.wav|speaker1|Este es otro audio del primer hablante.
#@markdown * wavs/speaker2audio1.wav|speaker2|Esto es lo que dice el segundo hablante en el primer audio.
#@markdown * wavs/speaker2audio2.wav|speaker2|Este es otro audio del segundo hablante.
#@markdown * ...

#@markdown #### Y así sucesivamente. Además, la transcripción debe estar en formato <font color="orange">**.csv (UTF-8 sin BOM)**
#@markdown ---
%cd /content/dataset
from google.colab import files
!rm /content/dataset/metadata.csv
listfn, length = files.upload().popitem()
if listfn != "metadata.csv":
  !mv "$listfn" metadata.csv
%cd ..

In [ ]:
#@markdown # <font color="pink"> **3. Preprocesar el dataset.** 🔄

import os
#@markdown ### En primer lugar, seleccione el idioma de su conjunto de datos. <br> (Está disponible para español los siguientes: Español y Español lationamericano.)
language = "Español" #@param ["Català", "Dansk", "Deutsch", "Ελληνικά", "English (British)", "English (U.S.)", "Español", "Español (latinoamericano)", "Suomi", "Français", "ქართული", "hindy", "Icelandic", "Italiano", "қазақша", "नेपाली", "Nederlands", "Norsk", "Polski", "Português (Brasil)", "Русский", "Svenska", "украї́нська", "Tiếng Việt", "简体中文"]
#@markdown ---
# language definition:
languages = {
    "Català": "ca",
    "Dansk": "da",
    "Deutsch": "de",
    "Ελληνικά": "grc",
    "English (British)": "en",
    "English (U.S.)": "en-us",
    "Español": "es",
    "Español (latinoamericano)": "es-419",
    "Suomi": "fi",
    "Français": "fr",
    "hindy": "hi",
    "Icelandic": "is",
    "Italiano": "it",
    "ქართული": "ka",
    "қазақша": "kk",
    "नेपाली": "ne",
    "Nederlands": "nl",
    "Norsk": "nb",
    "Polski": "pl",
    "Português (Brasil)": "pt-br",
    "Русский": "ru",
    "Svenska": "sv",
    "украї́нська": "uk",
    "Tiếng Việt": "vi-vn-x-central",
    "简体中文": "yue"
}

def _get_language(code):
    return languages[code]

final_language = _get_language(language)
#@markdown ### Elige un nombre para tu modelo:
model_name = "Test" #@param {type:"string"}
#@markdown ---
# output:
#@markdown ###Elige la carpeta de trabajo: (se recomienda guardar en Drive)

#@markdown La carpeta de trabajo se utilizará en el preprocesamiento, pero también en el entrenamiento del modelo.
output_path = "/content/drive/MyDrive/colab/piper" #@param {type:"string"}
output_dir = output_path+"/"+model_name
if not os.path.exists(output_dir):
  os.makedirs(output_dir)
#@markdown ---
#@markdown ### Elige el formato del dataset:
dataset_format = "ljspeech" #@param ["ljspeech", "mycroft"]
#@markdown ---
#@markdown ### ¿Se trata de un conjunto de datos de un solo hablante? Si no es así, desmarca la casilla:
single_speaker = True #@param {type:"boolean"}
if single_speaker:
  force_sp = " --single-speaker"
else:
  force_sp = ""
#@markdown ---
#@markdown ###Seleccione la frecuencia de muestreo del dataset:
sample_rate = "22050" #@param ["16000", "22050"]
#@markdown ---
%cd /content/piper/src/python
#@markdown ###¿Quieres entrenar utilizando esta frecuencia de muestreo, pero tus audios no la tienen?
#@markdown ¡El remuestreador te ayuda a hacerlo rápidamente!
resample = False #@param {type:"boolean"}
if resample:
  !python resample.py --input_dir "/content/dataset/wavs" --output_dir "/content/dataset/wavs_resampled" --output_sr {sample_rate} --file_ext "wav"
  !mv /content/dataset/wavs_resampled/* /content/dataset/wavs
#@markdown ---

!python -m piper_train.preprocess \
  --language {final_language} \
  --input-dir /content/dataset \
  --output-dir "{output_dir}" \
  --dataset-format {dataset_format} \
  --sample-rate {sample_rate} \
  {force_sp}

In [ ]:
#@markdown # <font color="pink"> **4. Ajustes.** 🧰
import json
import ipywidgets as widgets
from IPython.display import display
from google.colab import output
import os
#@markdown ### Seleccione la acción para entrenar este conjunto de datos:

#@markdown * La opción de continuar un entrenamiento se explica por sí misma. Si has entrenado previamente un modelo con colab gratuito, se te ha acabado el tiempo y estás considerando entrenarlo un poco más, esto es ideal para ti. Sólo tienes que establecer los mismos ajustes que estableciste cuando entrenaste este modelo por primera vez.
#@markdown * La opción para convertir un modelo de un solo hablante en un modelo multihablante se explica por sí misma, y para ello es importante que hayas procesado un conjunto de datos que contenga texto y audio de todos los posibles hablantes que quieras entrenar en tu modelo.
#@markdown * La opción finetune se utiliza para entrenar un conjunto de datos utilizando un modelo preentrenado, es decir, entrenar sobre esos datos. Esta opción es ideal si desea entrenar un conjunto de datos muy pequeño (se recomiendan más de cinco minutos).
#@markdown * La opción entrenar desde cero construye características como el diccionario y la forma del habla desde cero, y esto puede tardar más en converger. Para ello, se recomiendan horas de audio (8 como mínimo) que tengan una gran colección de fonemas.
action = "finetune" #@param ["Continue training", "convert single-speaker to multi-speaker model", "finetune", "train from scratch"]
#@markdown ---
if action == "Continue training":
    if os.path.exists(f"{output_dir}/lightning_logs/version_0/checkpoints/last.ckpt"):
        ft_command = f'--resume_from_checkpoint "{output_dir}/lightning_logs/version_0/checkpoints/last.ckpt" '
        print(f"Continuing {model_name}'s training at: {output_dir}/lightning_logs/version_0/checkpoints/last.ckpt")
    else:
        raise Exception("Training cannot be continued as there is no checkpoint to continue at.")
elif action == "finetune":
    if os.path.exists(f"{output_dir}/lightning_logs/version_0/checkpoints/last.ckpt"):
        raise Exception("Oh no! You have already trained this model before, you cannot choose this option since your progress will be lost, and then your previous time will not count. Please select the option to continue a training.")
    else:
        ft_command = '--resume_from_checkpoint "/content/pretrained.ckpt" '
elif action == "convert single-speaker to multi-speaker model":
    if not single_speaker:
        ft_command = '--resume_from_single_speaker_checkpoint "/content/pretrained.ckpt" '
    else:
        raise Exception("This dataset is not a multi-speaker dataset!")
else:
    ft_command = ""
if action== "convert single-speaker to multi-speaker model" or action == "finetune":
    try:
        with open('/content/piper/notebooks/pretrained_models.json') as f:
            pretrained_models = json.load(f)
        if final_language in pretrained_models:
            models = pretrained_models[final_language]
            model_options = [(model_name, model_name) for model_name, model_url in models.items()]
            model_dropdown = widgets.Dropdown(description = "Choose pretrained model", options=model_options)
            download_button = widgets.Button(description="Download")
            def download_model(btn):
                model_name = model_dropdown.value
                model_url = pretrained_models[final_language][model_name]
                print("Downloading pretrained model...")
                if model_url.startswith("1"):
                    !gdown -q "{model_url}" -O "/content/pretrained.ckpt"
                elif model_url.startswith("https://drive.google.com/file/d/"):
                    !gdown -q "{model_url}" -O "/content/pretrained.ckpt" --fuzzy
                else:
                    !wget -q "{model_url}" -O "/content/pretrained.ckpt"
                model_dropdown.close()
                download_button.close()
                output.clear()
                if os.path.exists("/content/pretrained.ckpt"):
                    print("Model downloaded!")
                else:
                    raise Exception("Couldn't download the pretrained model!")
            download_button.on_click(download_model)
            display(model_dropdown, download_button)
        else:
            raise Exception(f"There are no pretrained models available for the language {final_language}")
    except FileNotFoundError:
        raise Exception("The pretrained_models.json file was not found.")
else:
    print("Warning: this model will be trained from scratch. You need at least 8 hours of data for everything to work decent. Good luck!")
#@markdown ### Elige el tamaño del lote basándose en este conjunto de datos:
batch_size = 12 #@param {type:"integer"}
#@markdown ---
validation_split = 0.01
#@markdown ### Elige la calidad para este modelo:

#@markdown * x-low - 16Khz audio, 5-7M params
#@markdown * medium - 22.05Khz audio, 15-20 params
#@markdown * high - 22.05Khz audio, 28-32M params
quality = "medium" #@param ["high", "x-low", "medium"]
#@markdown ---
#@markdown ### Elige la calidad para este modelo: ¿Cada cuántas épocas quieres autoguardar los puntos de control de entrenamiento?
#@markdown Cuanto mayor sea tu conjunto de datos, debes establecer este intervalo de guardado en un valor menor, ya que las épocas pueden progresar durante más tiempo.
checkpoint_epochs = 5 #@param {type:"integer"}
#@markdown ---
#@markdown ### Intervalo de pasos para generar muestras de audio del modelo:
log_every_n_steps = 1000 #@param {type:"integer"}
#@markdown ---
#@markdown ### Número de épocas para el entrenamiento.
max_epochs = 10000 #@param {type:"integer"}
#@markdown ---

In [ ]:
#@markdown # <font color="pink"> **5. Entrenar.** 🏋️‍♂️
#@markdown Ejecuta esta celda para entrenar tu modelo. Si es posible, se guardarán algunas muestras de audio durante el entrenamiento en la carpeta de salida.

get_ipython().system(f'''
python -m piper_train \
--dataset-dir "{output_dir}" \
--accelerator 'gpu' \
--devices 1 \
--batch-size {batch_size} \
--validation-split {validation_split} \
--num-test-examples 2 \
--quality {quality} \
--checkpoint-epochs {checkpoint_epochs} \
--log_every_n_steps {log_every_n_steps} \
--max_epochs {max_epochs} \
{ft_command}\
--precision 32
''')

# ¿Has terminado el entrenamiento y quieres probar el modelo?

* ¡Si quieres ejecutar este modelo en cualquier software que Piper integre o en la misma app de Piper, exporta tu modelo usando el [cuaderno exportador de modelos](https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_model_exporter.ipynb)!
* Si quieres probar este modelo ahora mismo antes de exportarlo al formato soportado por Piper. ¡Prueba tu last.ckpt generado con [este cuaderno](https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_inference_(ckpt).ipynb)!